# What is Selenium?

## 1

What environment do we use?

See [What is Docker?](What%20is%20Docker.ipynb)

Run Selenium in Docker?


In [1]:
var importer = require('../Core');
var path = require('path');
var writeFileSync = require('fs').writeFileSync;

// add some run commands to the bash script
var bashToRun = (code) => code.split('\n').filter(n => n.trim().length > 0).map(l => 'RUN ' + l.trim())
    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');

// create a selenium Dockerfile with a vnc connection
var seleniumDocker = (outputFile) => {
    return importer.interpretAll([
        'run selenium',
        // add some extra services
        'linux dev tools',
        'vnc html',
        'vnc docker'
    ]).then(r => {
        // convert some results to Docker RUN commands instead of bash
        r[1].code = bashToRun(r[1].code);
        r[2].code = bashToRun(r[2].code);
        return r.map(r => r.code).join('\n');
    })
    .then(r => {
        // save the Dockerfile
        writeFileSync(outputFile, r);
        return r;
    });
};
module.exports = seleniumDocker;
seleniumDocker;


[Function: seleniumDocker]

How to set up selenium server?

In [9]:
var importer = require('../Core');
var path = require('path');

var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');
var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');

var execCmd;
var getSeleniumServer = (name = 'act-selenium') => {
    try {
        fs.mkdirSync(DOWNLOAD_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') { throw err; }
    }
    try {
        fs.mkdirSync(PROFILE_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') { throw err; }
    }
    try {
        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));
    } catch (err) {
        if (err.code != 'ENOENT') { throw err; }
    }
    var DOCKERFILE = path.resolve(path.join(__dirname, 'Dockerfile'));
    return seleniumDocker(DOCKERFILE)
        .then(() => importer.interpretAll('spawn child process'))
        .then(r => {
            execCmd = eval("'use strict';" + r[0].code);
            return execCmd('docker ps -a');
        })
        .then(r => {
            if(r[0].indexOf(name) > -1) {
                return execCmd('docker stop ' + name)
                    .then(r => new Promise(resolve => 
                        setTimeout(() => resolve(r), 1000)))
                    .then(() => execCmd('docker rm ' + name));
            }
        })
        .then(r => {
            var build = 'docker build -t ' + name + ' "' 
                + path.dirname(DOCKERFILE) + '"\n'
                + 'docker run --shm-size=3g -d '
                + '--name ' + name + ' ' 
                + '-p 8888:8888 '
                + '-p 6080:6080 '
                + '-p 5900:5900 '
                + '-p 4444:4444 '
                + '-p 4200:4200 '
                + '-p 3000:3000 '
                // TODO: add profile dir back in when permissions works on windows
                + '-v "' + DOWNLOAD_DIR + '":/data/downloads ' 
                + name + '\n';
            return execCmd(build)
        })
        .then(r => new Promise(resolve => setTimeout(() => resolve(r), 6000)));
};
module.exports = getSeleniumServer;
getSeleniumServer;



[Function: getSeleniumServer]

In [10]:
$$.async();
getSeleniumServer()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]


[ 'Sending build context to Docker daemon  159.2kB\r\r\nStep 1/24 : FROM selenium/standalone-chrome-debug\n ---> 593e47f379db\nStep 2/24 : EXPOSE 4200\n ---> Using cache\n ---> 25a779671ae2\nStep 3/24 : EXPOSE 4444\n ---> Using cache\n ---> 5c88188ae19e\nStep 4/24 : EXPOSE 3000\n ---> Using cache\n ---> 1ec30ab0541d\nStep 5/24 : ENV DBUS_SESSION_BUS_ADDRESS /dev/null\n ---> Using cache\n ---> c54c8757dcff\nStep 6/24 : ENV CHROME_USER_DATA_DIR /usr/profile\n ---> Using cache\n ---> 8de81f8efbb0\nStep 7/24 : WORKDIR /home/seluser\n ---> Using cache\n ---> 599c3d2be27e\nStep 8/24 : USER root\n ---> Using cache\n ---> 6ee27da9e04a\nStep 9/24 : RUN mkdir /usr/profile\n ---> Using cache\n ---> bf371b4cbb04\nStep 10/24 : RUN mkdir /usr/downloads\n ---> Using cache\n ---> d88e2e11064a\nStep 11/24 : RUN chown seluser:seluser -R /usr/profile\n ---> Using cache\n ---> cda4b62a4895\nStep 12/24 : RUN chown seluser:seluser -R /usr/downloads\n ---> Using cache\n ---> a430b2924546\nStep 13/24 : RUN ch

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser


In [1]:

var urlEncode = (obj) => {
    return Object.keys(obj).map(k => encodeURIComponent(k) + '=' + encodeURIComponent(obj[k])).join('&');
}

var vncIframe = (options = {
    password: 'secret',
    host: 'localhost',
    port: 6080,
    autoconnect: true,
    resize: 'downscale',
    view_only: true,
    reconnect: true}) => `
<div style="display:block; width:100%; padding-bottom:70%;position:relative;">
<iframe id="vnc"
style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" 
src="//local:6080/vnc.html?${urlEncode(options)}"></iframe></div>
`;
module.exports = vncIframe;
vncIframe;



[Function: vncIframe]

In [2]:
$$.mime({'text/html': vncIframe()});


## 3

How to run a selenium cell on the Docker machine?



In [3]:
var importer = require('../Core');

var runSeleniumCell = (search) => {
    var client, getCredentials;
    return importer.interpretAll(['webdriver client', 'decrypt password'])
        .then(r => {
            getCredentials = r[1].runInNewContext();
            return r[0].runInNewContext();
        })
        .then(r => client = r())
        .then(() => importer.interpretAll([search]))
        .then(r => r[0].runFresherInNewContext({client, getCredentials}))
};
module.exports = runSeleniumCell;
runSeleniumCell;



[Function: runSeleniumCell]

In [4]:
$$.async()
runSeleniumCell('test docker selenium')
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


Initializing webdriver on localhost
Already imported
[ 'import notebook.ipynb', 'interpret all notebooks.ipynb' ]


Test docker selenium?

In [8]:
if(typeof client !== 'undefined') {
    client.url('https://purchasesprint.actops.com');
    client.click('[href*="/auth"], [routerlink*="/auth"]');
    module.exports = client;
}



TODO:

Auto fill fields in a live browser using the 

Transfer login state to current browser?

